## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder(categories='auto')
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:
classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(32,kernel_size=(3,3),input_shape=(32,32,3),activation='relu', padding = 'SAME'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3),input_shape=(32,32,3),activation='relu', padding = 'SAME'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(activation="relu", units=100)) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(activation="softmax", units=10))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 13s 257us/step - loss: 1.4057 - acc: 0.5101
Epoch 2/100
50000/50000 [==============================] - 11s 222us/step - loss: 0.9566 - acc: 0.6611
Epoch 3/100
50000/50000 [==============================] - 11s 225us/step - loss: 0.7905 - acc: 0.7229
Epoch 4/100
50000/50000 [==============================] - 11s 225us/step - loss: 0.6614 - acc: 0.7675
Epoch 5/100
50000/50000 [==============================] - 11s 221us/step - loss: 0.5389 - acc: 0.8124
Epoch 6/100
50000/50000 [==============================] - 11s 219us/step - loss: 0.4121 - acc: 0.8556
Epoch 7/100
50000/50000 [==============================] - 11s 219us/step - loss: 0.3115 - acc: 0.8927
Epoch 8/100
50000/50000 [==============================] - 11s 220us/step - loss: 0.2260 - acc: 0.9232
Epoch 9/100
50000/50000 [==============================] - 11s 221us/step - loss: 0.1664 - acc: 0.9428
Epoch 10/100
50000/50000 [==============================] - 11s 222us/ste

50000/50000 [==============================] - 11s 219us/step - loss: 0.0169 - acc: 0.9944
Epoch 80/100
50000/50000 [==============================] - 11s 219us/step - loss: 0.0106 - acc: 0.9963
Epoch 81/100
50000/50000 [==============================] - 11s 220us/step - loss: 0.0140 - acc: 0.9958
Epoch 82/100
50000/50000 [==============================] - 11s 221us/step - loss: 0.0217 - acc: 0.9930
Epoch 83/100
50000/50000 [==============================] - 11s 219us/step - loss: 0.0190 - acc: 0.9943
Epoch 84/100
50000/50000 [==============================] - 11s 219us/step - loss: 0.0151 - acc: 0.9953
Epoch 85/100
50000/50000 [==============================] - 11s 219us/step - loss: 0.0155 - acc: 0.9952
Epoch 86/100
50000/50000 [==============================] - 11s 218us/step - loss: 0.0131 - acc: 0.9958
Epoch 87/100
50000/50000 [==============================] - 11s 220us/step - loss: 0.0159 - acc: 0.9950
Epoch 88/100
50000/50000 [==============================] - 11s 222us/step - 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[6.9916683e-01, 4.5193663e-15, 1.3894260e-01, 1.6173224e-01,
        7.9431439e-09, 8.1049554e-20, 6.4695339e-11, 7.4610448e-15,
        1.5835201e-04, 5.8552488e-18]], dtype=float32)

In [6]:
classifier.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 123us/step


[3.6246087682723998, 0.649]